# EVA3D: Compositional 3D Human Generation from 2D Image Collections

Authored by Fangzhou Hong, Zhaoxi Chen, Yushi Lan, Liang Pan, Ziwei Liu

This colab notebook provides a quick demo of EVA3D with the model trained on DeepFashion.

Useful links:

1. Github Repo: https://github.com/hongfz16/EVA3D
2. Project Page: https://hongfz16.github.io/projects/EVA3D.html
3. arXiv Link: https://arxiv.org/abs/2210.04888

In [ ]:
# @title Licensed under the S-Lab License

# S-Lab License 1.0

# Copyright 2022 S-Lab

# Redistribution and use for non-commercial purpose in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
# 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
# 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
# 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
# 4. In the event that redistribution and/or use for commercial purpose in source or binary forms, with or without modification is required, please contact the contributor(s) of the work.


In [1]:
# @title Setup Everything

!nvidia-smi
!git clone https://github.com/hongfz16/EVA3D.git
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
!pip install -r EVA3D/requirements.txt

Tue Nov 21 23:44:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# @title Download Models
%cd EVA3D
from download_models import download_pretrained_models
download_pretrained_models()

/content/EVA3D


100%|██████████| 160M/160M [00:07<00:00, 21.7MB/s]


100%|██████████| 160M/160M [00:06<00:00, 25.2MB/s]


100%|██████████| 160M/160M [00:07<00:00, 22.8MB/s]


100%|██████████| 158M/158M [00:04<00:00, 34.7MB/s]


Register and download SMPL models [here](https://smpl.is.tue.mpg.de/). Put the downloaded models in the folder smpl_models. Only the neutral one is needed. The folder structure should look like

```
./
├── ...
└── smpl_models/
    ├── smpl/
        └── SMPL_NEUTRAL.pkl
```

In [3]:
%cd EVA3D
from download_datasets import download_deepfashion
download_deepfashion()

[Errno 2] No such file or directory: 'EVA3D'
/content/EVA3D


100%|██████████| 4.50G/4.50G [01:31<00:00, 48.9MB/s]


In [4]:
!pip install torch torchvision torchaudio

In [5]:
!bash scripts/train_deepfashion_512x256.sh

+ python -m torch.distributed.launch --nproc_per_node 8 --master_port=21837 train_deepfashion.py --batch 1 --chunk 1 --expname train_deepfashion_512x256 --dataset_path datasets/DeepFashion --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --r1 300 --voxhuman_name eva3d_deepfashion --random_flip --eikonal_lambda 0.5 --small_aug --iter 1000000 --adjust_gamma --gamma_lb 20 --min_surf_lambda 1.5 --deltasdf --gaussian_weighted_sampler --sampler_std 15 --N_samples 28
/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*******

In [6]:
!bash scripts/demo_deepfashion_512x256.sh

+ python generation_demo.py --batch 1 --chunk 1 --expname 512x256_deepfashion --dataset_path demodataset --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --voxhuman_name eva3d_deepfashion --deltasdf --N_samples 28 --ckpt 420000 --identities 1 --truncation_ratio 0.5
/usr/local/lib/python3.10/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0% 0/1 [00:00<?, ?it/s]Marching Cube: 0/5
Marching Cube: 1/5
Marching Cube: 2/5
Marching Cube: 3/5
Marching Cube: 4/5
/usr/local/lib/python3.10/dist-packages/pytorch3d/io/obj_io.py:542: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")
/usr/local/lib/python3.10/dist-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is

In [ ]:
# @title Start the Genenration!
!python generation_demo.py --batch 1 --chunk 1 --expname 512x256_deepfashion --dataset_path demodataset --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --voxhuman_name eva3d_deepfashion --deltasdf --N_samples 28 --ckpt 420000 --identities 1 --truncation_ratio 0.5
from IPython.display import Image
Image('evaluations/512x256_deepfashion/iter_0420000/random_angles/images_paper_fig/0000000.png')

/usr/local/lib/python3.10/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Traceback (most recent call last):
  File "/content/EVA3D/generation_demo.py", line 238, in <module>
    g_ema = Generator(opt.model, opt.rendering, full_pipeline=False, voxhuman_name=opt.model.voxhuman_name).to(device)
  File "/content/EVA3D/model.py", line 879, in __init__
    self.renderer = VoxHuman_Class(renderer_opt, smpl_cfgs, out_im_res=tuple(model_opt.renderer_spatial_output_dim), style_dim=self.style_dim)
  File "/content/EVA3D/eva3d_deepfashion.py", line 279, in __init__
    self.smpl_model = init_smpl(
  File "/content/EVA3D/smpl_utils.py", line 43, in init_smpl
    smpl_model = build_layer(
  File "/usr/local/lib/python3.10/dist-packages/smplx/body_models.py", line 

FileNotFoundError: ignored

FileNotFoundError: ignored

<IPython.core.display.Image object>